# MOR(Merge-on-Read) 동작 원리 실습

이 노트북에서는 Iceberg의 **Merge-on-Read(MOR)** 전략이 실제로 어떻게 동작하는지 파일 수준에서 관찰합니다.  
특히 **Delete File**의 생성과 역할에 집중합니다.

## MOR(Merge-on-Read)란?

MOR은 COW의 "파일 전체 재작성" 문제를 해결하기 위한 전략입니다.

### 핵심 원리
전체 데이터 파일을 다시 쓰는 대신, **삭제 파일(Delete File)**에 어떤 레코드를 무시할지 기록합니다.

### 삭제(DELETE) 시 동작
- 기존 데이터 파일은 **그대로 유지**
- 삭제할 레코드의 위치가 **Delete File**에 기록됨

### 업데이트(UPDATE) 시 동작
1. 이전 레코드의 위치가 **Delete File**에 추가
2. 업데이트된 레코드만 **새 데이터 파일**에 작성

### 읽기(READ) 시 동작
- 데이터 파일과 Delete File을 **조정(merge)**하여 최종 결과 반환
- 즉, 삭제 표시된 레코드를 제외하고 읽음

### 장점 vs 단점
- **장점**: 쓰기가 빠름 (파일 전체를 재작성하지 않으므로)
- **단점**: 읽기가 느림 (Delete File과의 병합 작업 필요)

```
DELETE 시:
┌─────────────────┐    ┌─────────────────┐
│  데이터 파일       │    │  Delete File     │
│  (1000행, 그대로)  │ +  │  (삭제할 행 위치)   │
│  ✅ 파일 유지      │    │  ✅ 새로 생성      │
└─────────────────┘    └─────────────────┘

UPDATE 시:
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  기존 데이터 파일   │    │  Delete File     │    │  새 데이터 파일    │
│  (그대로 유지)     │ +  │  (기존 행 위치)    │ +  │  (변경된 행만)     │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, snapshot_tree, diff_tree, count_files, total_size

In [ ]:
spark = create_spark_session()

TABLE_NAME = "demo.lab.mor_orders"
TABLE_PATH = "/home/jovyan/data/warehouse/lab/mor_orders"

## MOR 테이블 생성

모든 쓰기 모드(delete, update, merge)를 `merge-on-read`로 설정합니다.

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

spark.sql(f"""
CREATE TABLE {TABLE_NAME} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='merge-on-read',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read'
)
""")

print(f"테이블 생성 완료: {TABLE_NAME}")

# 설정 확인
props = spark.sql(f"SHOW TBLPROPERTIES {TABLE_NAME}").collect()
for row in props:
    if 'write' in row['key']:
        print(f"  {row['key']} = {row['value']}")

---
## 실험 1: INSERT — 초기 데이터 적재

100건의 주문 데이터를 삽입합니다. INSERT는 COW와 동일하게 동작합니다.

In [ ]:
before_insert = snapshot_tree(TABLE_PATH)

# 100건 INSERT
orders = generate_orders(num_records=100, seed=42)
df = to_spark_df(spark, orders)
df.writeTo(TABLE_NAME).append()

after_insert = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("INSERT 후 변경 사항:")
print("=" * 60)
diff_tree(before_insert, after_insert)

In [ ]:
print("INSERT 후 테이블 트리 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

print(f"\n파일 수: {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")
print(f"총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")

### 관찰 포인트 — INSERT

- INSERT는 COW/MOR 구분 없이 동일하게 동작합니다
- 데이터 파일만 생성되었고, Delete File은 없습니다

---
## 실험 2: DELETE — Delete File 생성 관찰

`status='cancelled'`인 주문을 삭제합니다.  
MOR에서는 데이터 파일을 재작성하지 않고 **Positional Delete File**을 생성합니다!

In [ ]:
# DELETE 대상 확인
cancelled_count = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE status = 'cancelled'").collect()[0][0]
print(f"DELETE 대상 (status='cancelled'): {cancelled_count}건")

In [ ]:
before_delete = snapshot_tree(TABLE_PATH)

# DELETE 실행
spark.sql(f"DELETE FROM {TABLE_NAME} WHERE status = 'cancelled'")

after_delete = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("DELETE 후 변경 사항 (MOR):")
print("=" * 60)
diff_tree(before_delete, after_delete)

In [ ]:
print("DELETE 후 테이블 트리 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

print(f"\n파일 수 (parquet): {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")
print(f"총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")

### 관찰 포인트 — DELETE (MOR)

- **기존 데이터 파일은 그대로** 유지되었습니다!
- 대신 **Positional Delete File**이 새로 생성되었습니다
- COW였다면 데이터 파일이 통째로 재작성되었을 텐데, MOR은 작은 Delete File만 추가합니다
- 이것이 MOR의 쓰기 성능이 빠른 이유입니다

---
## 실험 3: UPDATE — Delete File + 새 데이터 파일

`status='pending'`인 주문을 `status='refunded'`로 변경합니다.  
MOR의 UPDATE는 두 가지를 동시에 수행합니다:
1. 기존 행의 위치를 **Delete File**에 기록
2. 변경된 행을 **새 데이터 파일**에 작성

In [ ]:
# UPDATE 대상 확인
pending_count = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE status = 'pending'").collect()[0][0]
print(f"UPDATE 대상 (status='pending'): {pending_count}건")

In [ ]:
before_update = snapshot_tree(TABLE_PATH)

# UPDATE 실행
spark.sql(f"UPDATE {TABLE_NAME} SET status = 'refunded' WHERE status = 'pending'")

after_update = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("UPDATE 후 변경 사항 (MOR):")
print("=" * 60)
diff_tree(before_update, after_update)

In [ ]:
print("UPDATE 후 테이블 트리 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

print(f"\n파일 수 (parquet): {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")

### 관찰 포인트 — UPDATE (MOR)

- **Delete File**이 추가 생성되었습니다 (기존 행의 위치 기록)
- **새 데이터 파일**도 생성되었습니다 (변경된 행만 포함)
- 기존 데이터 파일은 **역시 그대로** 유지됩니다
- 즉, UPDATE = Delete File + 새 데이터 파일 조합

---
## Delete File 상세 분석

MOR의 핵심인 Delete File을 자세히 들여다봅니다.

In [ ]:
# files 메타데이터 테이블에서 Delete File 확인
# content 컬럼: 0=데이터, 1=positional deletes, 2=equality deletes
print("전체 파일 목록 (데이터 + Delete):")
spark.sql(f"""
    SELECT 
        content,
        CASE content 
            WHEN 0 THEN 'DATA'
            WHEN 1 THEN 'POSITIONAL DELETE'
            WHEN 2 THEN 'EQUALITY DELETE'
        END as file_type,
        file_path,
        record_count,
        file_size_in_bytes
    FROM {TABLE_NAME}.files
    ORDER BY content, file_path
""").show(truncate=50)

In [ ]:
# Delete File만 필터링
print("Delete File 상세:")
delete_files_df = spark.sql(f"""
    SELECT file_path, record_count, file_size_in_bytes
    FROM {TABLE_NAME}.files
    WHERE content > 0
""")
delete_files_df.show(truncate=False)

In [ ]:
# Delete File을 pyarrow로 직접 읽어보기
import pyarrow.parquet as pq

delete_files = delete_files_df.collect()
if delete_files:
    first_delete_file = delete_files[0]['file_path']
    print(f"Delete File 경로: {first_delete_file}\n")
    
    try:
        table = pq.read_table(first_delete_file)
        print(f"스키마: {table.schema}")
        print(f"레코드 수: {len(table)}")
        print(f"\n내용 (처음 10행):")
        print(table.to_pandas().head(10))
    except Exception as e:
        print(f"읽기 실패: {e}")
        print("(Delete File 형식이 표준 Parquet과 다를 수 있습니다)")
else:
    print("Delete File이 없습니다.")

---
## Delete File 유형 설명

Iceberg에는 두 가지 유형의 Delete File이 있습니다:

### 1. Positional Delete (위치 기반 삭제)
- **컬럼**: `file_path` + `pos` (행 번호)
- **원리**: "이 파일의 N번째 행을 무시하라"
- **비유**: 영화관 좌석번호로 찾기 — "3열 5번 좌석의 사람"
- Spark에서 MOR 사용 시 기본적으로 생성되는 유형

```
Positional Delete File 내용:
┌───────────────────────────┬─────┐
│ file_path                 │ pos │
├───────────────────────────┼─────┤
│ data/part-00001.parquet   │  3  │
│ data/part-00001.parquet   │  7  │
│ data/part-00001.parquet   │ 15  │
└───────────────────────────┴─────┘
```

### 2. Equality Delete (값 기반 삭제)
- **컬럼**: 삭제 조건에 해당하는 컬럼값
- **원리**: "이 값과 일치하는 모든 행을 무시하라"
- **비유**: 군중에서 빨간 모자를 쓴 사람 모두 찾기 — "빨간 모자인 사람"
- Spark에서는 거의 사용되지 않음 (주로 Flink 등에서 활용)

```
Equality Delete File 내용:
┌──────────┐
│ order_id │
├──────────┤
│    42    │
│    57    │
│    89    │
└──────────┘
```

In [ ]:
# 스냅샷 히스토리 확인
print("전체 스냅샷 히스토리:")
spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").show(truncate=False)

---
## 정리: MOR(Merge-on-Read) 핵심 요약

| 항목 | 설명 |
|------|------|
| **쓰기 방식** | Delete File에 삭제할 행 위치를 기록, 기존 파일 유지 |
| **Delete File** | Positional Delete (file_path + pos) 생성 |
| **읽기 성능** | 느림 — 데이터 파일 + Delete File 병합 필요 |
| **쓰기 성능** | 빠름 — 파일 전체 재작성 없음 |
| **적합한 워크로드** | 쓰기 중심, 업데이트/삭제가 빈번한 경우 |

> **핵심**: MOR에서는 **Delete File만 추가하고 기존 데이터 파일은 그대로** 유지됩니다.  
> 읽을 때 데이터 파일과 Delete File을 **병합(merge)**하여 최종 결과를 만듭니다.  
> 다음 노트북에서 COW와 MOR을 직접 비교해봅니다.

In [ ]:
spark.stop()
print("Spark 세션 종료")